<a href="https://colab.research.google.com/github/DenisAksyan/1/blob/main/%D0%90%D0%BA%D1%81%D0%B5%D0%BD%D0%BE%D0%B2%D1%81%D0%BA%D0%B8%D0%B9_%D0%94_%D0%90__21_1_Grid_%D0%97%D0%B0%D0%B4%D0%B0%D1%87%D0%B0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.2/466.2 kB 12.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-4.0.1-cp310-cp310-linux_x86_64.whl size=4266348 sha256=3442bb5d47f4208adb87bbce17029ec4c36ab26ba77604c7354ffbde46f29698
  Stored in directory: /root/.cache/pip/wheels/3c/ca/13/13218a83854023ccec184e3af482f0f038b434aa32c19afee8
Successfully built mpi4py


In [2]:
from mpi4py import MPI
import numpy as np

In [3]:
def greedy_schedule(tasks, time_slots):
    # Сортируем задачи по приоритету (по убыванию)
    tasks.sort(key=lambda x: x['priority'])

    # Сортируем временные слоты по времени
    time_slots.sort(key=lambda x: x['duration'])

    schedule = {}

    for task in tasks:
        for slot in time_slots:
            # Проверяем, достаточно ли времени в слоте для выполнения задачи
            if task['duration'] <= slot['duration']:
                # Записываем задачу в расписание
                schedule[slot['start_time']] = task['name']
                if slot['duration'] != 0:
                    slot['start_time'] = slot['start_time'] + ' '
                # Уменьшаем доступное время в слоте
                slot['duration'] -= task['duration']
                # Если слот полностью занят, убираем его из списка
                if slot['duration'] == 0:
                    time_slots.remove(slot)

                break  # Переходим к следующей задаче

    return schedule

def main():
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    # Пример входных данных (для простоты, они одинаковые для всех процессов)
    tasks = [
        {'name': 'Task 1', 'duration': 2, 'priority': 1},
        {'name': 'Task 2', 'duration': 1, 'priority': 3},
        {'name': 'Task 3', 'duration': 3, 'priority': 2},
        {'name': 'Task 4', 'duration': 2, 'priority': 4},
        {'name': 'Task 5', 'duration': 1, 'priority': 5}
    ]

    time_slots = [
        {'start_time': '09:00', 'duration': 3},
        {'start_time': '10:00', 'duration': 2},
        {'start_time': '11:00', 'duration': 4}
    ]

    # Разделение задач между процессами
    num_tasks = len(tasks)
    tasks_per_process = num_tasks // size
    remainder = num_tasks % size

    start_index = rank * tasks_per_process + min(rank, remainder)
    end_index = start_index + tasks_per_process + (1 if rank < remainder else 0)

    local_tasks = tasks[start_index:end_index]

    # Каждый процесс выполняет жадный алгоритм на своем наборе задач
    local_schedule = greedy_schedule(local_tasks, time_slots.copy())

    # Сбор результатов от всех процессов
    all_schedules = comm.gather(local_schedule, root=0)

    if rank == 0:
        # Объединение расписаний
        final_schedule = {}
        for schedule in all_schedules:
            final_schedule.update(schedule)

        print("Itog")
        for start_time, task_name in final_schedule.items():
            print(f"{start_time}: {task_name}")
        print(final_schedule.items())

In [4]:
if __name__ == "__main__":
    main()

Itog
10:00: Task 1
09:00: Task 3
11:00: Task 2
11:00 : Task 4
11:00  : Task 5
dict_items([('10:00', 'Task 1'), ('09:00', 'Task 3'), ('11:00', 'Task 2'), ('11:00 ', 'Task 4'), ('11:00  ', 'Task 5')])


mpiexec -n 3 python 1.py